The dataset used in this project is the [Home Equity dataset](https://www.kaggle.com/datasets/ajay1735/hmeq-data/data). This dataset contains baseline and loan performance information for 5,960 recent home equity loans. It contains the following columns:
<ul>
<li>BAD: 1 = client defaulted on loan 0 = loan repaid</li>
<li>LOAN: Amount of the loan request</li>
<li>MORTDUE: Amount due on existing mortgage</li>
<li>VALUE: Value of current property</li>
<li>REASON: DebtCon = debt consolidation HomeImp = home improvement</li>
<li>JOB: Six occupational categories</li>
<li>YOJ: Years at present job</li>
<li>DEROG: Number of major derogatory reports</li>
<li>DELINQ: Number of delinquent credit lines</li>
<li>CLAGE: Age of oldest trade line in months</li>
<li>NINQ: Number of recent credit lines</li>
<li>CLNO: Number of credit lines</li>
<li>DEBTINC: Debt-to-income ratio</li></ul>